<a href="https://colab.research.google.com/github/paulaRMachado/PUC-RIO-DTSC-MVP2/blob/main/PUC_MVP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MVP Análise de Dados e Boas Práticas**

**Nome**: Paula R A Machado

**Matrícula**: 405205000163

**Dataset**: Indian Liver Patient Records

# Descrição do Problema

O conjunto de dados *Indian Liver Patient Records* contém informações laboratoriais e clínicas de pacientes, com o objetivo principal de prever a presença ou ausência de doenças hepáticas. Para isso, o dataset reúne medidas bioquímicas como níveis enzimáticos (ALT, AST), bilirrubina, proteínas totais, albumina, além de dados demográficos como idade e sexo. A proposta deste trabalho é investigar essas características e construir um modelo capaz de identificar precocemente pacientes que necessitam de avaliação médica especializada.

## Hipóteses do Problema

As hipóteses que tracei são as seguintes:

* Pacientes com níveis elevados de bilirrubina e enzimas hepáticas (AST, ALT) têm maior probabilidade de apresentar doença hepática.

* Existe relação entre o nível de albumina e a presença da doença hepática?

* Homens são mais frequentemente diagnosticados com doença hepática do que mulheres?

* Pacientes mais velhos tendem a apresentar um risco maior para o desenvolvimento de doença hepática?

* Medidas como a proporção entre albumina e globulina são bons indicadores para diferenciar pacientes saudáveis de doentes?

## Tipo de Problema

Este é um problema de **classificação supervisionada**. Dado um conjunto de características (comprimento e largura da sépala, comprimento e largura da pétala), o objetivo é prever a qual das três classes (espécies de Íris) uma determinada flor pertence.

## Seleção de Dados

O dataset *Indian Liver Patient Records* foi obtido do repositório Kaggle e contém informações laboratoriais e demográficas de pacientes com e sem doenças hepáticas. Por se tratar de um dataset real e não-curado, foi necessário carregar o arquivo CSV bruto e inspecionar a estrutura para verificar a qualidade dos dados antes da análise. Nessa etapa inicial, selecionamos todos os atributos disponíveis para uso, pois eles são potencialmente relevantes para o problema e não há necessidade de filtragem prévia de colunas.

## Atributos do Dataset

O dataset Indian Liver Patient Records contém 583 registros, sendo 416 de pacientes COM doença hepática e 167 de pacientes SEM doença hepática. Há 11 atributos:

- **Age**: Idade do paciente
- **Gender**: Sexo do Paciente
- **Total Bilirubin**: Bilirrubina Total (mg/dL)
- **Direct Bilirubin**: Bilirrubina Conjugada (mg/dL)
- **Alkaline Phosphotase**: Fosfatase Alcalina (UI/L)
- **Alamine Aminotransferase**: Alanina Aminotransferase (UI/L)
- **Aspartate Aminotransferase**: Aspartato Aminotransferase (UI/L)
- **Total Protiens**: Proteínas Totais (g/dL)
- **Albumin**: Albumina (g/dL)
- **Albumin and Globulin Ratio**: Razão Albumina/Globulina
- **Label**: Classificação (com doença hepática ou saudável)

In [1]:
import pandas as pd

NÂO ESQUECER DE SEPARAR TRAIN E TEST

Requisitos e composição da nota:
* (1,0 pt) Execução sem erros:o notebook deve poder ser executado pelo professor do início ao fim sem erros no Google Colab.
* (2,0 pts) Documentação consistente:utilize blocos de texto que expliquem textualmente cada etapa e cada decisão do seu código, contando uma história completa e compreensível, do início ao fim.
* (1,0 pt) Código limpo:seu código deve estar legível e organizado. Devem ser utilizadas as boas práticas de codificação vistas nas disciplinas Programação orientada a objetose Engenharia de software para ciência de dados, mas não é necessário que você crie classes no seu código.
* (2,0 pts) Análise de dados:após cada gráfico, você deverá escrever um parágrafo resumindo os principais achados, analisando os resultados e levantando eventuais pontos de atenção.
* (2,0 pts) Checklist:você deverá responder às perguntas (aplicáveis ao seu dataset) da checklist fornecida, utilizando-a como guia para o desenvolvimento do trabalho.
* (2,0 pts) Capricho e qualidade do trabalho como um todo.

**Análise de Dados**
Objetivo: entender a informação disponível.

Estatísticas descritivas:

Quantos atributos e instâncias existem?
Quais são os tipos de dados dos atributos?
Verifique as primeiras linhas do dataset. Algo chama a atenção?
Há valores faltantes, discrepantes ou inconsistentes?
Faça um resumo estatístico dos atributos com valor numérico (mínimo, máximo, mediana, moda, média, desvio-padrão e número de valores ausentes). O que você percebe?
Visualizações:

Verifique a distribuição de cada atributo. O que você percebe? Dica: esta etapa pode dar ideias sobre a necessidade de transformações na etapa de preparação de dados (por exemplo, converter atributos de um tipo para outro, realizar operações de discretização, normalização, padronização, etc.).
Se for um problema de classificação, verifique a distribuição de frequência das classes. O que você percebe? Dica: esta etapa pode indicar a possível necessidade futura de balanceamento de classes.
Analise os atributos individualmente ou de forma combinada, usando os gráficos mais apropriados.

Checklist sugerida:
Definição do problema
Objetivo: entender e descrever claramente o problema que está sendo resolvido.


1. Qual é a descrição do problema?
2. Este é um problema de aprendizado supervisionado ou não supervisionado?
3. Que premissas ou hipóteses você tem sobre o problema?
4. Que restrições ou condições foram impostas para selecionar os dados?
5. Defina cada um dos atributos do dataset. Dicionário de dados!